## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavaratti,IN,10.5669,72.642,81.34,67,5,6.31,clear sky
1,1,Provideniya,RU,64.3833,-173.300,-8.93,90,5,4.81,clear sky
2,2,Baykit,RU,61.6700,96.370,1.11,94,59,2.55,broken clouds
3,3,Jutai,BR,-5.1833,-68.900,72.84,98,68,0.78,broken clouds
4,4,Vaini,TO,-21.2000,-175.200,86.16,79,40,11.50,scattered clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavaratti,IN,10.5669,72.6420,81.34,67,5,6.31,clear sky
4,4,Vaini,TO,-21.2000,-175.2000,86.16,79,40,11.50,scattered clouds
9,9,Busselton,AU,-33.6500,115.3333,83.08,36,0,2.98,clear sky
12,12,Hobart,AU,-42.8794,147.3294,82.26,46,75,18.41,broken clouds
14,14,Alofi,NU,-19.0595,-169.9187,82.29,78,96,14.97,overcast clouds
21,21,Bengkulu,ID,-3.8004,102.2655,83.91,67,93,5.75,overcast clouds
25,25,Butaritari,KI,3.0707,172.7902,82.36,75,100,12.57,overcast clouds
32,32,Port Alfred,ZA,-33.5906,26.8910,76.57,93,99,11.86,light rain
39,39,San Policarpo,PH,12.1791,125.5072,79.86,81,99,15.90,overcast clouds
40,40,Mahebourg,MU,-20.4081,57.7000,83.25,83,40,11.50,scattered clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                145
City                   145
Country                145
Lat                    145
Lng                    145
Max Temp               145
Humidity               145
Cloudiness             145
Wind Speed             145
Current Description    145
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                145
City                   145
Country                145
Lat                    145
Lng                    145
Max Temp               145
Humidity               145
Cloudiness             145
Wind Speed             145
Current Description    145
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavaratti,IN,81.34,clear sky,10.5669,72.6420,
4,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,
9,Busselton,AU,83.08,clear sky,-33.6500,115.3333,
12,Hobart,AU,82.26,broken clouds,-42.8794,147.3294,
14,Alofi,NU,82.29,overcast clouds,-19.0595,-169.9187,
21,Bengkulu,ID,83.91,overcast clouds,-3.8004,102.2655,
25,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,
32,Port Alfred,ZA,76.57,light rain,-33.5906,26.8910,
39,San Policarpo,PH,79.86,overcast clouds,12.1791,125.5072,
40,Mahebourg,MU,83.25,scattered clouds,-20.4081,57.7000,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.count()

City                   145
Country                145
Max Temp               145
Current Description    145
Lat                    145
Lng                    145
Hotel Name             145
dtype: int64

In [24]:
# Print Clean Hotel df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavaratti,IN,81.34,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands
4,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Busselton,AU,83.08,clear sky,-33.6500,115.3333,Observatory Guest House
12,Hobart,AU,82.26,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
14,Alofi,NU,82.29,overcast clouds,-19.0595,-169.9187,Taloa Heights
21,Bengkulu,ID,83.91,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
25,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
32,Port Alfred,ZA,76.57,light rain,-33.5906,26.8910,The Halyards Hotel
39,San Policarpo,PH,79.86,overcast clouds,12.1791,125.5072,Agapita M. Nicart
40,Mahebourg,MU,83.25,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>CurrentDescription</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))